<a href="https://colab.research.google.com/github/Jez-Carter/Learning_Development_Statistics/blob/master/python/mlcroissant/recipes/tfds_croissant_builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a model with Croissant 🥐, Hugging Face 🤗 and TFDS

[TensorFlow Datasets](https://www.tensorflow.org/datasets/overview) (in short, TFDS) is an established library to handle downloading and preparing data efficiently and deterministically.

TFDS is framework-agnostic: it can generate datasets by constructing a `tf.data.Dataset`, a `np.array` or a [`ArrayRecord`](https://github.com/google/array_record) data source, for use with TensorFlow, Jax, PyTorch, and other Machine Learning frameworks.

TFDS has recently introduced a `CroissantBuilder`, which defines a TFDS dataset based on a Croissant 🥐 metadata file.

## Setup



Let's install and import the needed dependencies:

In [34]:
%%capture --no-display
# Install mlcroissant from the source
!apt-get install -y python3-dev graphviz libgraphviz-dev pkg-config
!pip install "git+https://github.com/${GITHUB_REPOSITORY:-mlcommons/croissant}.git@${GITHUB_HEAD_REF:-main}#subdirectory=python/mlcroissant&egg=mlcroissant[dev]"
!pip install array_record
!pip install tfds-nightly
!pip install tensorflow
!pip install torch
!apt-get install tree

In [35]:
%%capture --no-display
import json
import os

from etils import epath
import mlcroissant as mlc
import requests
import tensorflow_datasets as tfds
import torch
from tqdm import tqdm

local_croissant_file = epath.Path("/tmp/croissant.json")
data_dir = "/tmp/croissant"

## Download the Croissant JSON-LD file

To initialize a `CroissantBuilder` in TFDS, we need a Croissant 🥐 file describing a dataset.
In this notebook, we will create a TFDS `CroissantBuilder` for [fashion_mnist](https://huggingface.co/datasets/fashion_mnist), a popular dataset for computer vision.

In [3]:
api_url = "https://huggingface.co/api/datasets/fashion_mnist/croissant"

# Download the JSON and write it to `local_croissant_file`.
response = requests.get(api_url, headers=None).json()
with local_croissant_file.open("w") as f:
  jsonld = json.dumps(response, indent=2)
  f.write(jsonld)
  print(jsonld)

{
  "@context": {
    "@language": "en",
    "@vocab": "https://schema.org/",
    "citeAs": "cr:citeAs",
    "column": "cr:column",
    "conformsTo": "dct:conformsTo",
    "cr": "http://mlcommons.org/croissant/",
    "data": {
      "@id": "cr:data",
      "@type": "@json"
    },
    "dataBiases": "cr:dataBiases",
    "dataCollection": "cr:dataCollection",
    "dataType": {
      "@id": "cr:dataType",
      "@type": "@vocab"
    },
    "dct": "http://purl.org/dc/terms/",
    "extract": "cr:extract",
    "field": "cr:field",
    "fileProperty": "cr:fileProperty",
    "fileObject": "cr:fileObject",
    "fileSet": "cr:fileSet",
    "format": "cr:format",
    "includes": "cr:includes",
    "isLiveDataset": "cr:isLiveDataset",
    "jsonPath": "cr:jsonPath",
    "key": "cr:key",
    "md5": "cr:md5",
    "parentField": "cr:parentField",
    "path": "cr:path",
    "personalSensitiveInformation": "cr:personalSensitiveInformation",
    "recordSet": "cr:recordSet",
    "references": "cr:reference

## Build the TFDS dataset

A `CroissantBuilder` takes as input a Croissant 🥐 file, and a list of `RecordSet` names to generate. Each `RecordSet` will correspond to a separated [`BuilderConfig`](https://www.tensorflow.org/datasets/api_docs/python/tfds/core/BuilderConfig).

In [36]:
builder = tfds.core.dataset_builders.CroissantBuilder(
    jsonld="/tmp/croissantSpikeZip.json",
    record_set_ids=["rs-abberfraw"],
    file_format='array_record',
    data_dir="/tmp/croissant_ukceh",
)

In [ ]:
builder = tfds.core.dataset_builders.CroissantBuilder(
    jsonld=local_croissant_file,
    record_set_ids=["fashion_mnist"],
    file_format='array_record',
    data_dir=data_dir,
)

  -  [Metadata(fashion_mnist)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
  -  [Metadata(fashion_mnist)] Property "https://schema.org/datePublished" is recommended, but does not exist.
  -  [Metadata(fashion_mnist)] Property "https://schema.org/version" is recommended, but does not exist.


Our `CroissantBuilder` uses the information contained in the Croissant 🥐 file to initialize the TFDS dataset's [documentation](https://www.tensorflow.org/datasets/api_docs/python/tfds/core/DatasetInfo), which we can explore using the [`DatasetBuilder.info`](https://www.tensorflow.org/datasets/api_docs/python/tfds/core/DatasetBuilder#info) method:

In [37]:
print(f"Dataset's description:\n{builder.info.description}\n")
print(f"Dataset's citation:\n{builder.info.citation}\n")
print(f"Dataset's features:\n{builder.info.features}")


Dataset's description:
This data contains values of bare sand area, modelled wind speed, aspect and slope at a 2.5 m spatial resolution for four UK coastal dune fields, Abberfraw (Wales), Ainsdale (England), Morfa Dyffryn (Wales), Penhale (England). Data is stored as a .csv file. Data is available for 620,756.25 m2 of dune at Abberfraw, 550,962.5 m2 of dune at Ainsdale, 1,797,756.25 m2 of dune at Morfa Dyffryn and 2,275,056.25 m2 of dune at Penhale. All values were calculated from aerial imagery and digital terrain models collected between 2014 and 2016.

Dataset's citation:
Smyth, T.A.G. (2022). Bare sand, wind speed, aspect and slope at four English and Welsh coastal sand dunes, 2014-2016. NERC EDS Environmental Information Data Centre. https://doi.org/10.5285/972599af-0cc3-4e0e-a4dc-2fab7a6dfc85

Dataset's features:
FeaturesDict({
    'Aspect': float32,
    'BareSand_it1': float32,
    'Slope': float32,
    'WindSpeed': float32,
    'X': float32,
    'Y': float32,
    'id': int64,
}

In [18]:
print(f"Dataset's features:\n{builder.info.features}")

Dataset's features:
FeaturesDict({
    'Aspect': float32,
    'BareSand_it1': float32,
    'Slope': float32,
    'WindSpeed': float32,
    'X': float32,
    'Y': float32,
    'id': int64,
})


In [ ]:
print(f"Dataset's description:\n{builder.info.description}\n")
print(f"Dataset's citation:\n{builder.info.citation}\n")
print(f"Dataset's features:\n{builder.info.features}")

# ...

Dataset's description:
Dataset Card for FashionMNIST







		Dataset Summary


Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and… See the full description on the dataset page: https://huggingface.co/datasets/zalando-datasets/fashion_mnist.

Dataset's citation:


Dataset's features:
FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8, description=Image column 'image' from the Hugging Face parquet file.),
    'label': int64,
    'split': Text(shape=(), dtype=string),
})


We can now generate and materialize the TFDS dataset on disk:

In [38]:
%%capture --no-display
builder.download_and_prepare()

`download_and_prepare` downloads the data and prepares the dataset specifically for ML. For instance, it uses an ML-optimized data format. You can read more [in the documentation](https://www.tensorflow.org/datasets/tfless_tfds). Let's inspect it on disk:

In [39]:
!tree "/tmp/croissant_ukceh/"

/tmp/croissant_ukceh/
├── downloads
│   └── extracted
└── dunes_data
    └── rs_abberfraw
        └── 1.0.0
            ├── dataset_info.json
            ├── dunes_data-default.array_record-00000-of-00001
            ├── features.json
            └── LICENSE

5 directories, 4 files


In [ ]:
!tree {data_dir}/

/tmp/croissant/
├── downloads
│   └── extracted
└── zalando_datasets__fashion_mnist
    └── fashion_mnist
        └── 1.0.0
            ├── dataset_info.json
            ├── features.json
            ├── LICENSE
            ├── zalando_datasets__fashion_mnist-test.array_record-00000-of-00001
            └── zalando_datasets__fashion_mnist-train.array_record-00000-of-00001

5 directories, 5 files


The command above outputs a dictionary of data sources with a train/test split:

In [ ]:
mnist_builder = tfds.builder("mnist")
mnist_info = mnist_builder.info
mnist_builder.download_and_prepare()
datasets = mnist_builder.as_dataset()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.LU0K0P_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.LU0K0P_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [11]:
ds = builder.as_data_source()

In [40]:
len(ds['default'])

99321

In [43]:
import tensorflow as tf
# Convert the datasource into a tf.data.Dataset
dataset = tf.data.Dataset.from_generator(
    lambda: (example for example in ds),  # Generator function
    output_signature={
        "feature1": tf.TensorSpec(shape=(), dtype=tf.float32),
        "feature2": tf.TensorSpec(shape=(), dtype=tf.float32),
        "label": tf.TensorSpec(shape=(), dtype=tf.int32),
    }
)

In [50]:
import tensorflow as tf

# Define TensorFlow dataset from generator
dataset = tf.data.Dataset.from_generator(
    lambda: (example for example in ds['default']),
    output_signature={
        "id": tf.TensorSpec(shape=(), dtype=tf.int32),
        "X": tf.TensorSpec(shape=(), dtype=tf.float32),
        "Y": tf.TensorSpec(shape=(), dtype=tf.float32),
        "Aspect": tf.TensorSpec(shape=(), dtype=tf.float32),
        "Slope": tf.TensorSpec(shape=(), dtype=tf.float32),
        "WindSpeed": tf.TensorSpec(shape=(), dtype=tf.float32),
        "BareSand_it1": tf.TensorSpec(shape=(), dtype=tf.float32),
    }
)


In [51]:
# Print a few records from tf.data.Dataset
for example in dataset.take(5):  # Take the first 5 records
    print(example)


{'id': <tf.Tensor: shape=(), dtype=int32, numpy=140520>, 'X': <tf.Tensor: shape=(), dtype=float32, numpy=235566.25>, 'Y': <tf.Tensor: shape=(), dtype=float32, numpy=368331.25>, 'Aspect': <tf.Tensor: shape=(), dtype=float32, numpy=287.2237854003906>, 'Slope': <tf.Tensor: shape=(), dtype=float32, numpy=0.9244160652160645>, 'WindSpeed': <tf.Tensor: shape=(), dtype=float32, numpy=1.1475861072540283>, 'BareSand_it1': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>}
{'id': <tf.Tensor: shape=(), dtype=int32, numpy=216762>, 'X': <tf.Tensor: shape=(), dtype=float32, numpy=235916.25>, 'Y': <tf.Tensor: shape=(), dtype=float32, numpy=368126.25>, 'Aspect': <tf.Tensor: shape=(), dtype=float32, numpy=136.97999572753906>, 'Slope': <tf.Tensor: shape=(), dtype=float32, numpy=21.083030700683594>, 'WindSpeed': <tf.Tensor: shape=(), dtype=float32, numpy=1.300819754600525>, 'BareSand_it1': <tf.Tensor: shape=(), dtype=float32, numpy=52.0>}
{'id': <tf.Tensor: shape=(), dtype=int32, numpy=202054>, 'X': <tf.Ten

In [44]:
dataset

<_FlatMapDataset element_spec={'feature1': TensorSpec(shape=(), dtype=tf.float32, name=None), 'feature2': TensorSpec(shape=(), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int32, name=None)}>

In [41]:
%%timeit
ds['default'][0]

The slowest run took 4.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1.1 ms ± 594 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%%timeit
ds['default'][20000]

532 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
for example in ds['default']:
  print(example)

Streaming output truncated to the last 5000 lines.
{'Aspect': 252.51174926757812, 'BareSand_it1': 23.0, 'Slope': 14.210899353027344, 'WindSpeed': 0.7148095369338989, 'X': 235673.75, 'Y': 367778.75, 'id': 164133}
{'Aspect': 101.546630859375, 'BareSand_it1': 0.0, 'Slope': 6.771269798278809, 'WindSpeed': 0.7031749486923218, 'X': 236001.25, 'Y': 368198.75, 'id': 235229}
{'Aspect': 130.03184509277344, 'BareSand_it1': 53.0, 'Slope': 16.386398315429688, 'WindSpeed': 0.5836663842201233, 'X': 235893.75, 'Y': 368148.75, 'id': 211857}
{'Aspect': 169.67698669433594, 'BareSand_it1': 0.0, 'Slope': 4.3842573165893555, 'WindSpeed': 1.1496926546096802, 'X': 235746.25, 'Y': 368723.75, 'id': 179531}
{'Aspect': 259.0135803222656, 'BareSand_it1': 0.0, 'Slope': 20.147178649902344, 'WindSpeed': 0.586242139339447, 'X': 235433.75, 'Y': 368201.25, 'id': 111740}
{'Aspect': 179.61961364746094, 'BareSand_it1': 0.0, 'Slope': 2.9246506690979004, 'WindSpeed': 1.2124335765838623, 'X': 236041.25, 'Y': 368438.75, 'id': 

KeyboardInterrupt: 

In [ ]:
datasets

{Split('train'): <_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>,
 Split('test'): <_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}

In [22]:
print(data_source['default'][6])

{'Aspect': 299.36572265625, 'Slope': 12.624134063720703, 'WindSpeed': 1.2469048500061035, 'X': 235726.25, 'Y': 368731.25, 'id': 175176}


In [23]:
def dataset_to_dataframe(dataset):
    features_list = []
    labels_list = []

    for example in dataset:  # Iterate through dataset
        features = example['image'].numpy().flatten()  # Flatten image
        label = example['label'].numpy()  # Extract label

        features_list.append(features)
        labels_list.append(label)

    df = pd.DataFrame(features_list)
    df["label"] = labels_list  # Add labels column

    return df

# Convert the dataset
df = dataset_to_dataframe(data_source['default'])
print(df.head())  # Check the data

KeyError: 'image'

In [ ]:
# ds_all_dict = builder.as_dataset()
# assert isinstance(ds_all_dict, dict)
print(datasets.keys())  # ==> ['test', 'train', 'unsupervised']


dict_keys([Split('train'), Split('test')])


In [19]:
train,test = builder.as_data_source(split=['default[:75%]','default[75%:]'])

In [21]:
print(len(train),len(test))

74491 24830


In [23]:
batch_size = 128
train_sampler = torch.utils.data.RandomSampler(train, num_samples=5_000)
train_loader = torch.utils.data.DataLoader(
    train,
    sampler=train_sampler,
    batch_size=batch_size,
)
test_loader = torch.utils.data.DataLoader(
    test,
    sampler=None,
    batch_size=batch_size,
)

In [29]:
from torch.utils.data import DataLoader, Dataset

# Convert data to PyTorch Dataset
class DuneDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = list(dataset)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]
        x = torch.tensor([example['Aspect'], example['Slope'], example['WindSpeed']], dtype=torch.float32)
        y = torch.tensor(example['BareSand_it1'], dtype=torch.float32)
        return x, y

train_dataset = DuneDataset(train)
test_dataset = DuneDataset(test)

# DataLoader for PyTorch
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [30]:
# Define the PyTorch Linear Regression Model
class LinearRegressor(torch.nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressor, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

# Initialize model, loss, and optimizer
model = LinearRegressor(input_dim=3)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


In [28]:
for

TypeError: 'DataLoader' object is not subscriptable

In [31]:
# Train the model
epochs = 50
for epoch in range(epochs):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(x_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: nan
Epoch 10, Loss: nan
Epoch 20, Loss: nan
Epoch 30, Loss: nan
Epoch 40, Loss: nan


In [33]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluate the model
y_true, y_preds = [], []
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        y_pred = model(x_batch).squeeze()
        y_preds.extend(y_pred.numpy())
        y_true.extend(y_batch.numpy())

mse = mean_squared_error(y_true, y_preds)
r2 = r2_score(y_true, y_preds)

print("Model Performance:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

ValueError: Input contains NaN.

In [24]:
train.Aspect

AttributeError: 'ArrayRecordDataSource' object has no attribute 'Aspect'

In [ ]:
builder.info.splits['default[:75%]'].num_examples  # 7_500 (also works with slices)

74491

In [ ]:
train_ds, test_ds = tfds.load('mnist', split=['train', 'test[:50%]'])

In [ ]:
data

ArrayRecordDataSource(name=dunes_data, split='default[:75%]', decoders=None)

In [ ]:
, split='train[:75%]')

In [ ]:
data

{'default': ArrayRecordDataSource(name=dunes_data, split='default', decoders=None)}

In [ ]:
data = builder.as_data_source()

In [ ]:
train.info

AttributeError: 'ArrayRecordDataSource' object has no attribute 'info'

In [ ]:
train, test = builder.as_data_source(split=['train', 'test'])

## Train a model

TFDS can be used with TensorFlow, JAX and PyTorch, because it supports many data loaders like [tf.data](https://www.tensorflow.org/guide/data), [PyGrain](https://github.com/google/grain) and [PyTorch DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). For example, let's try with Torch:

In [ ]:
torch.utils.data.RandomSampler(data, num_samples=len(data))

In [ ]:
batch_size = 128
train_sampler = torch.utils.data.RandomSampler(train, num_samples=len(train))
train_loader = torch.utils.data.DataLoader(
    train,
    sampler=train_sampler,
    batch_size=batch_size,
)
test_loader = torch.utils.data.DataLoader(
    test,
    sampler=None,
    batch_size=batch_size,
)

In [ ]:
batch_size = 128
train_sampler = torch.utils.data.RandomSampler(train, num_samples=len(train))
train_loader = torch.utils.data.DataLoader(
    train,
    sampler=train_sampler,
    batch_size=batch_size,
)
test_loader = torch.utils.data.DataLoader(
    test,
    sampler=None,
    batch_size=batch_size,
)

DataLoaders can be fed in input of any ML pipeline. Let's try the example of a very simple example:

In [ ]:
class LinearClassifier(torch.nn.Module):
  def __init__(self, shape, num_classes):
    super(LinearClassifier, self).__init__()
    height, width, channels = shape
    self.classifier = torch.nn.Linear(height * width * channels, num_classes)

  def forward(self, image):
    image = image.view(image.size()[0], -1).to(torch.float32)
    return self.classifier(image)

shape = train[0]["image"].shape
num_classes = 10
model = LinearClassifier(shape, num_classes)
optimizer = torch.optim.Adam(model.parameters())
loss_function = torch.nn.CrossEntropyLoss()

print('Training...')
model.train()
for example in tqdm(train_loader):
  image = example['image']
  label = example['label']
  prediction = model(image)
  loss = loss_function(prediction, label)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

print('Testing...')
model.eval()
num_examples = 0
true_positives = 0
for example in tqdm(test_loader):
  image = example['image']
  label = example['label']
  prediction = model(image)
  num_examples += image.shape[0]
  predicted_label = prediction.argmax(dim=1)
  true_positives += (predicted_label == label).sum().item()
print(f'\nAccuracy: {true_positives/num_examples * 100:.2f}%')

Training...


100%|██████████| 469/469 [00:10<00:00, 43.92it/s]


Testing...


100%|██████████| 79/79 [00:02<00:00, 32.32it/s]


Accuracy: 74.04%


# Test

In [ ]:
train,test = builder.as_data_source(split=['default[:75%]','default[75%:]'])

In [ ]:
import tensorflow as tf

In [ ]:
# Convert data source to dataset
dataset = tf.data.Dataset.from_generator(
    lambda: data_source,
    output_signature={
        'modeled_wind_speed': tf.TensorSpec(shape=(), dtype=tf.float32),
        'aspect': tf.TensorSpec(shape=(), dtype=tf.float32),
        'slope': tf.TensorSpec(shape=(), dtype=tf.float32),
        'bare_sand_area': tf.TensorSpec(shape=(), dtype=tf.float32)
    }
)

In [ ]:
# Step 2: Data Preprocessing
def preprocess(features):
    x = tf.stack([features['modeled_wind_speed'], features['aspect'], features['slope']], axis=-1)
    y = features['bare_sand_area']
    return x, y

# Apply preprocessing
dataset = dataset.map(preprocess).batch(32).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [ ]:
# Split dataset into train and test sets
data_size = sum(1 for _ in dataset)
train_size = int(0.8 * data_size)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)


UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} NameError: name 'data_source' is not defined
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/dataset_ops.py", line 865, in get_iterator
    return self._iterators[iterator_id]
           ~~~~~~~~~~~~~~~^^^^^^^^^^^^^

KeyError: 1


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = n

In [ ]:
# Apply preprocessing
dataset = train.map(preprocess).batch(32).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)


AttributeError: 'ArrayRecordDataSource' object has no attribute 'map'